In [ ]:
!pip install datasets
!pip install faiss-gpu
!pip install langchain
!pip install langchain-community langchain-core

In [ ]:
!pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:
import pandas as pd
from datasets import load_dataset
import json
import os
import sys
import boto3
from botocore.client import Config
import faiss

In [ ]:
# will be using Titan Embedding model called form LangChain to generate embeddings of querry
from langchain_community.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# for Data Ingestion
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# For Vectore embedding and Vectore Store (Using Fiass DB fot embedding)
from langchain.vectorstores import faiss
from langchain_community.vectorstores import FAISS

## Prompt eng
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever



In [ ]:
dataset = load_dataset("hugginglearners/netflix-shows")
dataset = pd.DataFrame(dataset['train'])
# dataset.to_json('/content/drive/MyDrive/Netflix_Project/data.json')
dataset.head()

In [ ]:
# Upload the file to an S3 bucket.
session = boto3.Session()
s3 = session.resource('s3')
bucket = s3.Bucket('moviedatabucket')
# bucket.Object('dataset.json').put(Body=open('dataset.json', 'rb'))


In [ ]:
# Bedrock Client
session = boto3.Session(
    aws_access_key_id='AWS_ACCESS_KEY',
    aws_secret_access_key='AWS_SECRET_ACCESS_KEY',
    aws_session_token=boto3.client('sts').assume_role(
        RoleArn='arn:aws:iam::975050062872:role/ColabAccess',
        RoleSessionName='ColabSession'  # You can customize the session name
    )['Credentials']['SessionToken'],
)
region =session.region_name
print(region)

In [ ]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 3})

bedrock_client = boto3.client("bedrock-runtime", region_name = region)

# bedrock_client.create_evaluation_job()

In [ ]:
# Instance for Titan Embedding model from bedrock
bedrock_embedding = BedrockEmbeddings(model_id='amazon.titan-embed-text-v2:0',
                                      client = bedrock_client)

In [ ]:
# Implementing data Ingestion
obj = bucket.Object('data.json')
response = obj.get()
data = json.load(response['Body'])


In [ ]:
def data_ingest():
    loader = TextLoader('data.json', encoding='utf8')
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size  = 10000,
                                                   chunk_overlap = 500)
    doc_chunk =  text_splitter.split_documents(data)
    return doc_chunk

In [ ]:
# Vector Embedding and Vector store
def get_vectorestore(docs):
    vectore_store_fiass = FAISS.from_documents(docs,
                                               bedrock_embedding)
    vectore_store_fiass.save_local('faiss_index')

In [ ]:
# Create/call LLM

def get_llama():
  # Create Anthropic Model
    llm = Bedrock(model_id='meta.llama3-2-1b-instruct-v1:0',
                client=bedrock_client,
                model_kwargs={'max_gen_len': 300,'temperature': 0.3})
    return llm

In [ ]:
# Create Prompt template

prompt_temp = """
Human: Answer the question asked at the end, use the context given to you, if you dont have the answer say dont know, do not makeup some answer.
Keep the answer precies and brife don't make long lenthy answers. If asked for suggestion give top 3 suggestion saying you can suggest more if user want.
<context>
{context}
</context>
Question: {Question}
Assistant:
"""

prompt = PromptTemplate(template=prompt_temp, input_variables=['context', 'Question'])

In [ ]:
# Get response from LLM

def get_response_llm(llm, vectorestore_fiass, query):
    # Get Retrival object using AmazonRetrival from Langchain framework
    retriever = AmazonKnowledgeBasesRetriever(
          knowledge_base_id= vectorestore_fiass,
          retrieval_config={"vectorSearchConfiguration":
                            {"numberOfResults": 4,
                            'overrideSearchType': "SEMANTIC", # optional
                            }
                            },)

    # Get the context
    qa = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type='stuff',
      retriever=retriever,
      return_source_documents=True,
      chain_type_kwargs={'prompt':prompt}
    )
    answer = qa({'query': query})
    return answer['result']



In [ ]:
doc_chunks = data_ingest()
print(type(data_ingest()))
get_vectorestore(doc_chunks)

In [ ]:
# prompt: save a vectore on drive at given path
file_name = "vectore_store.pkl"

# Upload file to S3
bucket.Object(file_name).put(Body=open(file_name, "rb"))

In [ ]:
# Load the LLM
llm = get_llama()

# Ask a query
query = input("Ask your movie-related question: ")

# Get response
response = get_response_llm(llm, "faiss_index", query)  # if faiss_index is the path
print("Answer:", response)